# CS5242

## Load Data

In [1]:
from data import Data

train_data = Data(data_type='train')
test_data = Data(data_type='test')

print("Train data: x:"+ str(train_data.x.shape)+ ' y:'+str(train_data.y.shape))
print("Test data: x:"+ str(test_data.x.shape)+ ' y:'+str(test_data.y.shape))



Loading train data 18662/18662: 100%|█████████████████████████████████████████████████████████████ [ time left: 00:00 ]
Loading test data 6051/6051: 100%|████████████████████████████████████████████████████████████████ [ time left: 00:00 ]


Train data: x:(18662, 1000, 102) y:(18662, 1)
Test data: x:(6051, 1000, 102) y:(6051, 1)


## Model

In [2]:
import tensorflow as tf

model_name = 'lstm_pooling'

model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True),input_shape=(1000,102)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(1e-4,0.95,0.99),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1000, 256)         236544    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 253,057
Trainable params: 253,057
Non-trainable params: 0
_________________________________________________________________


## Save Checkpoint

In [3]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './checkpoints/' + model_name
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=20, 
    mode='auto', 
    restore_best_weights=True)

## Training

In [ ]:
history = model.fit(x=train_data.x, y=train_data.y, validation_split=0.2, epochs=100,batch_size=32,
                    callbacks=[checkpoint_callback,early_stopping_callback], verbose=2,shuffle=True)

Train on 14929 samples, validate on 3733 samples
Epoch 1/100
14929/14929 - 41s - loss: 0.1277 - accuracy: 0.8260 - val_loss: 0.0926 - val_accuracy: 0.8786
Epoch 2/100
14929/14929 - 38s - loss: 0.0723 - accuracy: 0.9092 - val_loss: 0.0751 - val_accuracy: 0.8995
Epoch 3/100
14929/14929 - 38s - loss: 0.0561 - accuracy: 0.9302 - val_loss: 0.0593 - val_accuracy: 0.9218
Epoch 4/100
14929/14929 - 38s - loss: 0.0484 - accuracy: 0.9407 - val_loss: 0.0580 - val_accuracy: 0.9212
Epoch 5/100
14929/14929 - 38s - loss: 0.0428 - accuracy: 0.9459 - val_loss: 0.0534 - val_accuracy: 0.9271
Epoch 6/100
14929/14929 - 38s - loss: 0.0395 - accuracy: 0.9516 - val_loss: 0.0520 - val_accuracy: 0.9269
Epoch 7/100
14929/14929 - 38s - loss: 0.0365 - accuracy: 0.9538 - val_loss: 0.0501 - val_accuracy: 0.9352
Epoch 8/100
14929/14929 - 38s - loss: 0.0337 - accuracy: 0.9586 - val_loss: 0.0469 - val_accuracy: 0.9408
Epoch 9/100
14929/14929 - 38s - loss: 0.0317 - accuracy: 0.9609 - val_loss: 0.0489 - val_accuracy: 0.93

## Plot Training History

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Restore Weights

In [ ]:
checkpoint_dir = './checkpoints/' + model_name
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
test_predict = model.predict(test_data.x)
print(test_predict.shape)



## Save Prediction to CSV

In [ ]:
import csv

output_file = 'result_' + model_name + '.csv'
with open(output_file, 'wt', newline='', encoding='utf-8') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(('Id', 'Predicted'))
    for id, predict in enumerate(test_predict):
        csv_writer.writerow((id, predict[0]))
print('Saved to '+'result_' + model_name + '.csv')